In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping
import seaborn as sns
from IPython.core.display_functions import display
from IPython.display import Audio
from livelossplot import PlotLossesKeras

import keras
from keras import layers
from keras.models import Sequential

from keras_preprocessing.image import img_to_array

import tensorflow as tf
from keras.layers import *

import os
from tqdm.notebook import tqdm
from pathlib import Path
import shutil


from music_plots import *
from sklearn.metrics import roc_auc_score
import numpy as np

2022-06-19 15:47:18.112409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-19 15:47:18.112455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/marco/repo/ML/venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)



<h1> Data management </h1>

In [2]:
tracks_df = load("data/tracks.csv")
genres_df = load("data/genres.csv")
features_df = load("data/features.csv")

In [3]:
tracks_df.head()
genres_df.head()
#features_df.head()

,#tracks,parent,title,top_level
genre_id,,,,
1,8693,38,Avant-Garde,38
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5


In [4]:
track_genres = tracks_df.xs('track', level=0, axis=1)['genre_top'].loc[features_df.dropna().index]
track_genres = track_genres.dropna()
features_df = features_df.loc[track_genres.index]

track_genres

track_id
2             Hip-Hop
3             Hip-Hop
5             Hip-Hop
10                Pop
134           Hip-Hop
             ...     
95823    Experimental
95831    Experimental
95866    Experimental
95868    Experimental
95871    Experimental
Name: genre_top, Length: 32823, dtype: category
Categories (16, object): ['Blues', 'Classical', 'Country', 'Easy Listening', ..., 'Pop', 'Rock', 'Soul-RnB', 'Spoken']

In [5]:
# genres dictionary
genres = {}
index = 0
for i in track_genres.unique():
    genres[i] = index
    index += 1
print(genres)

{'Hip-Hop': 0, 'Pop': 1, 'Rock': 2, 'Experimental': 3, 'Folk': 4, 'Jazz': 5, 'Electronic': 6, 'Spoken': 7, 'International': 8, 'Soul-RnB': 9, 'Blues': 10, 'Country': 11, 'Classical': 12, 'Old-Time / Historic': 13, 'Instrumental': 14, 'Easy Listening': 15}


In [6]:
print(len(track_genres))
print(len(features_df))
features_df.isna().any()
features_df

32823
32823


feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
95823        -0.555692 -0.338323  0.883295  0.424315  1.450894  0.070183   
95831        -0.612445  0.702932  0.776201  0.186624  0.580003  0.287438   
95866         3.528371  4.667782  5.247205  4.168351  3.276534  4.309053   
95868        -0.033556  0.271865  0.280602  0.560396  1.511805  0.710866   
95871         5.215266  4.964749  0.824476  1.733783  2.557416  3.057908   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
95823       0.716997  0.757623  0.300295  0.063043  ...  0.064474  0.012939   
95831      -0.037945 -1.151037 -1.074581  0.307664  ...  0.072024  0.011513   
95866       2.768949  3.115098  3.815794  3.940277  ...  0.031871  0.012202   
95868       1.451442  0.742910  0.739932  0.844851  ...  0.055034  0.012744   
95871       1.768616  3.665696  1.518494  1.523711  ...  0.058086  0.016176   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
95823       0.013397   6.294609  0.230469  0.039246  0.032715  0.004883   
95831       0.013774  -0.943706  0.441406  0.117571  0.106934  0.005371   
95866       0.012386   1.357928  0.400391  0.135901  0.143066  0.000000   
95868       0.013205  -0.223611  0.402344  0.173291  0.182617  0.000000   
95871       0.013788   0.166801  0.561035  0.211801  0.218262  0.000000   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
95823       1.909812  0.026319  
95831       0.452720  0.085089  
95866      -0.795436  0.055681  
95868      -0.706943  0.048163  
95871      -0.058994  0.087301  

[32823 rows x 518 columns]

<h1>SVM </h1>

In [7]:
# generate train and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_df, track_genres, test_size=0.4, random_state=42, stratify=track_genres)

In [8]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svm_model=Pipeline([
                             ('scaler',StandardScaler()),
                             ('svm_clf', SVC(kernel='rbf',class_weight='balanced',probability=True))
                             ])
svm_model.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(class_weight='balanced', probability=True))])

In [9]:
prediction = svm_model.predict(X_train)
proba_train= svm_model.predict_proba(X_train)[:,1]
print(proba_train)
print(f'AUC_train:{roc_auc_score(y_train,proba_train)}')
pd.DataFrame(prediction, columns=list(genres.keys()))

[7.32382300e-04 2.99199686e-05 3.52433334e-04 ... 5.51836512e-06
 3.05018469e-04 4.66074146e-04]


ValueError: multi_class must be in ('ovo', 'ovr')